# Week 4 Exercise: Python → C++ Optimizer + Complexity Note

This notebook converts a Python function into optimized C++ and provides complexity analysis and micro-optimizations.

**Output format:**
- C++ Code (as a code block)
- Complexity (time + space)
- Optimization Notes
- Example I/O


In [30]:
# Imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI


In [31]:
# Load environment variables (.env)
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print('No API key found. Please add OPENAI_API_KEY to your .env file.')
elif api_key.strip() != api_key:
    print('API key has leading/trailing whitespace. Please remove it.')
else:
    print('API key looks good!')

openai = OpenAI(
    api_key=api_key,
    base_url='https://openrouter.ai/api/v1',
)


API key looks good!


In [32]:
# Model selection
MODEL = 'gpt-4.1-mini'


In [33]:
# Python function input (replace with your own)
python_code = '''
def top_k_frequent(nums, k):
    counts = {}
    for n in nums:
        counts[n] = counts.get(n, 0) + 1
    return sorted(counts.keys(), key=lambda x: counts[x], reverse=True)[:k]
'''


In [34]:
SYSTEM_PROMPT = '''
You are a senior C++ engineer and performance tuner.
Convert Python code to optimized C++17.
You MUST return markdown with these exact headings (use ##):
## C++ Code
## Complexity
## Optimization Notes
## Example I/O
Rules:
- C++ code must be inside a single fenced code block with ```cpp.
- Complexity must include time and space.
- Optimization Notes must be 2-4 bullet points.
- Example I/O must show one sample input and output.
If you fail to follow the format exactly, the answer is invalid.
'''

USER_PROMPT_PREFIX = '''
Convert this Python function to optimized C++ and analyze it:

'''


In [35]:
def messages_for(code_text: str):
    return [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': USER_PROMPT_PREFIX + code_text},
    ]


In [36]:
# Post-process output to enforce headings + code block
def enforce_format(md: str) -> str:
    required = ['## C++ Code', '## Complexity', '## Optimization Notes', '## Example I/O']
    for h in required:
        if h not in md:
            md = h + '\n' + md
    # Ensure C++ code is fenced
    if '```cpp' not in md:
        # naive wrap: insert a cpp fence after the C++ Code heading
        parts = md.split('## C++ Code', 1)
        if len(parts) == 2:
            before, after = parts
            md = before + '## C++ Code\n```cpp\n' + after.strip() + '\n```'
    return md


In [37]:
# Non-streaming version
def convert_and_optimize(code_text: str) -> str:
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages_for(code_text),
        max_tokens=900,
    )
    return enforce_format(response.choices[0].message.content)


In [38]:
# Streaming version
def convert_and_optimize_stream(code_text: str):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=messages_for(code_text),
        stream=True,
        max_tokens=900,
    )
    response = ''
    display_handle = display(Markdown(''), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

    # Re-render final response with enforced formatting
    final_md = enforce_format(response)
    update_display(Markdown(final_md), display_id=display_handle.display_id)

convert_and_optimize_stream(python_code)


## C++ Code
```cpp
#include <vector>
#include <unordered_map>
#include <algorithm>

std::vector<int> top_k_frequent(const std::vector<int>& nums, int k) {
    std::unordered_map<int, int> counts;
    counts.reserve(nums.size());  // Reserve to reduce rehashing

    for (int n : nums) {
        ++counts[n];
    }

    std::vector<int> unique;
    unique.reserve(counts.size());
    for (const auto& kvp : counts) {
        unique.push_back(kvp.first);
    }

    std::nth_element(unique.begin(), unique.begin() + k, unique.end(),
        [&counts](int a, int b) { return counts[a] > counts[b]; });

    unique.resize(k);
    std::sort(unique.begin(), unique.end(),
        [&counts](int a, int b) { return counts[a] > counts[b]; });

    return unique;
}
```

## Complexity
- Time: O(N + M log k), where N is the size of `nums`, M is the number of unique elements (M ≤ N). Counting frequencies is O(N), `nth_element` is average O(M), resizing and sorting top k elements is O(k log k).
- Space: O(M) for storing frequency counts and the unique elements vector.

## Optimization Notes
- Used `unordered_map` for O(1) average frequency counting.
- Reserved capacity for the map and vector to minimize reallocations.
- Used `nth_element` to partially sort only the top k, improving performance over full sorting.
- Sorted only the top k elements to finalize order, thus reducing overhead.

## Example I/O
Input: nums = {1,1,1,2,2,3}, k = 2  
Output: {1, 2}

In [20]:
# Or use the non-streaming version:
# display(Markdown(convert_and_optimize(python_code)))


## C++ Code
```cpp
#include <vector>
#include <unordered_map>
#include <queue>
#include <algorithm>

std::vector<int> top_k_frequent(const std::vector<int>& nums, int k) {
    // Count frequencies
    std::unordered_map<int, int> counts;
    for (int n : nums) {
        ++counts[n];
    }

    // Use a min-heap to keep track of top k frequent elements
    // The heap will store pairs of (frequency, element)
    auto cmp = [](const std::pair<int,int>& a, const std::pair<int,int>& b) {
        return a.first > b.first; // min-heap based on frequency
    };
    std::priority_queue<std::pair<int,int>, std::vector<std::pair<int,int>>, decltype(cmp)> min_heap(cmp);

    for (const auto& kv : counts) {
        min_heap.emplace(kv.second, kv.first);
        if ((int)min_heap.size() > k) {
            min_heap.pop();
        }
    }

    // Extract results from the heap (in ascending frequency order)
    std::vector<int> result;
    result.reserve(k);
    while (!min_heap.empty()) {
        result.push_back(min_heap.top().second);
        min_heap.pop();
    }
    // Reverse to get descending frequency order
    std::reverse(result.begin(), result.end());
    return result;
}
```

## Complexity
- Time complexity: O(N log k), where N is the number of elements in nums. Counting takes O(N), and maintaining the heap of size k takes O(log k) per insert.
- Space complexity: O(N) for the frequency hash map and O(k) for the heap.

## Optimization Notes
- Using a min-heap of size k reduces sorting cost compared to sorting all unique elements.
- The hash map provides O(1) average time complexity for frequency counts.
- Reserving result vector capacity avoids repeated reallocations.
- The approach balances speed and memory for large inputs with many unique elements.

## Example I/O
Input:  
`nums = {1,1,1,2,2,3}, k = 2`

Output:  
`{1, 2}`  
Explanation: 1 appears 3 times, 2 appears 2 times, 3 appears once. Top 2 frequent are 1 and 2.